In [1]:
from langchain_teddynote import logging


logging.langsmith("runnable_study")

LangSmith 추적을 시작합니다.
[프로젝트명]
runnable_study


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

prompt = PromptTemplate.from_template('{num}의 10배는?')
llm = ChatOllama(model = 'llama3')

In [6]:
chain = prompt | llm 


chain.invoke({"num": 5})

AIMessage(content='😊\n\n5의 10배 means "10 times 5".\n\nTo calculate this, you would multiply 5 by 10:\n\n5 × 10 = 50\n\nSo, 5의 10배 is 50! 👍', response_metadata={'model': 'llama3', 'created_at': '2024-07-05T12:32:35.473384Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 11444025041, 'load_duration': 6840246458, 'prompt_eval_count': 17, 'prompt_eval_duration': 332232000, 'eval_count': 51, 'eval_duration': 4268218000}, id='run-1b9a5aca-90c5-4f98-87d4-ffa7ce60271a-0')

RunnalblePassthrough는 runnable 객체이며, invoke() 메소드를 사용하여 별도 실행 가능.

In [7]:
from langchain_core.runnables import RunnablePassthrough


RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [8]:
# 사용자의 입력을 전달
# key을 기억하고 입력하지 않아도 값만 전달할 수 있음.
runnable_chain  = {"num": RunnablePassthrough()} | prompt | llm

runnable_chain.invoke(10)

AIMessage(content='😊\n\n10의 10배는 100입니다. (The 10 times of 10 is 100.)', response_metadata={'model': 'llama3', 'created_at': '2024-07-05T12:35:41.159544Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3255931875, 'load_duration': 3512875, 'prompt_eval_count': 12, 'prompt_eval_duration': 1026820000, 'eval_count': 26, 'eval_duration': 2223382000}, id='run-9c22d046-5554-4dc5-94bc-4561f4839176-0')

RunnalblePassthrough.assign()
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합친다.

In [9]:
(RunnablePassthrough.assign(new_num = lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

## RunnableParallel

In [11]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성. 여러 Runnalble 인스턴스를 병렬로 실행 가능
runnalble = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달. 입력된 데이터 그대로 통과시키는 역할
    passed = RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthorough.assign을 사용하여 'mult' 람다 함수 할당. 입력된 딕셔너리의 'num'의 값 3배 증가
    extra = RunnablePassthrough.assign(mult = lambda x : x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수 전달. 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더함.
    modified = lambda x: x["num"] + 1, 
)
runnalble.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [16]:
from langchain_core.output_parsers import StrOutputParser
chain1 = (
    {"country": RunnablePassthrough()} 
    | PromptTemplate.from_template("{country} 의 수도는?")
    | llm
    | StrOutputParser()
    )

chain2 = (
    {"country": RunnablePassthrough()} 
    | PromptTemplate.from_template("{country} 의 면적은?")
    | llm
    | StrOutputParser()
    )


In [17]:
combined_chain = RunnableParallel(capitial = chain1, area = chain2)
combined_chain.invoke('대한민국')

{'capitial': '😊\n\nThe capital of South Korea is 서울특별시 (Seoul).',
 'area': '🇰🇷 The Republic of Korea, also known as South Korea, has a total area of approximately 100,363 square kilometers (38,750 square miles). 📍'}

## RunnableLambda

RunnableLambda를 사용하여 사용자 정의 함수를 맵핑 가능


In [43]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from datetime import datetime


def get_today(a):
    print(f"입력받은 변수 a의 값: {a}")
    return datetime.today().strftime("%b-%d")

get_today(None)

입력받은 변수 a의 값: None


'Jul-05'

In [44]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일도 작성해 주세요."
)

llm = ChatOllama(model = 'ollama-ko-0502')

chain = (
    {"today": RunnableLambda(get_today),"n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [45]:
chain.invoke(3)

입력받은 변수 a의 값: 3


'이름, 생년월일을 입력하십시오.\n1. 김정은(1994 년 1 월 8 일) 2. 이승기(1987 년 10 월 16 일) 3. 박지성(1981 년 2 월 25 일'

In [50]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일도 작성해 주세요."
)

llm = ChatOllama(model = 'ollama-ko-0502')

chain = (
    {"today": RunnableLambda(get_today),"n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)

In [51]:
chain.invoke({"n":3})

입력받은 변수 a의 값: {'n': 3}


'이름, 생년월일을 입력하십시오.\n1. 김정은(1994 년 1 월 8 일) 2. 이승기(1987 년 10 월 16 일) 3. 박지성(1981 년 2 월 25 일'

In [52]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOllama(model = 'ollama-ko-0502')

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [54]:
chain.invoke({'word1': 'hello','word2': 'world'})

AIMessage(content='5 와 25 를 더하면 30 이 됩니다. 5 와 25 의 합을 계산하는 방법은 다음과 같습니다. 먼저, 5 에서 1 을 빼면 4 가 되고, 25 에서 1 을 빼면 24 가 됩니다. 그리고 4 와 24 를 더하면 28 이 되고, 28 에서 2 를 빼면 26 이 되며, 26 에서 5 를 빼면 21 이 됩니다. 마지막으로 21 과 25 를 더하면 46 이 되고, 46 에서 1 을 빼면 45 가 되므로, 5 와 25 의 합은 45 가 됩니다', response_metadata={'model': 'ollama-ko-0502', 'created_at': '2024-07-05T13:37:11.978647Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 17561255750, 'load_duration': 4830334, 'prompt_eval_count': 15, 'prompt_eval_duration': 770515000, 'eval_count': 154, 'eval_duration': 16778002000}, id='run-a57faa01-bc48-43db-a510-cf47561359a3-0')